In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine

In [5]:
# Считываем все креды
src_host = os.environ.get('DB_SOURCE_HOST')
src_port = os.environ.get('DB_SOURCE_PORT')
src_username = os.environ.get('DB_SOURCE_USER')
src_password = os.environ.get('DB_SOURCE_PASSWORD')
src_db = os.environ.get('DB_SOURCE_NAME') 

dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [6]:
# Создадим соединения
src_conn = create_engine(f'postgresql://{src_username}:{src_password}@{src_host}:{src_port}/{src_db}')
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [7]:
TABLE = 'flats_table'
SQL = f'select * from {TABLE}'
data = pd.read_sql(SQL, dst_conn)

In [8]:
data.head()

,id,building_id,floor,kitchen_area,living_area,rooms,is_apartment,studio,total_area,price,build_year,building_type_int,latitude,longitude,ceiling_height,flats_count,floors_total,has_elevator
0,18634,18237,8,12.8,0.0,4,false,false,155.600006,49900000.0,2001,4,55.697655,37.506927,2.7,185,18,true
1,18635,19594,8,13.0,50.0,2,false,false,78.000000,31000000.0,2005,2,55.702728,37.509705,3.0,552,17,false
2,18636,861,4,12.0,67.0,3,false,false,88.000000,28500000.0,1938,1,55.804955,37.512344,3.2,228,8,true
3,18637,24197,8,25.0,0.0,3,false,false,88.000000,40000000.0,2018,2,55.770805,37.564213,3.0,1630,48,true
4,18638,22417,11,35.0,180.0,6,false,false,260.000000,199000000.0,2013,2,55.786572,37.617626,3.3,134,12,true


In [9]:
data.describe

<bound method NDFrame.describe of            id  building_id  floor  kitchen_area  living_area  rooms  \
0       18634        18237      8          12.8     0.000000      4   
1       18635        19594      8          13.0    50.000000      2   
2       18636          861      4          12.0    67.000000      3   
3       18637        24197      8          25.0     0.000000      3   
4       18638        22417     11          35.0   180.000000      6   
...       ...          ...    ...           ...          ...    ...   
141357  87989         5622      3           6.4    18.000000      1   
141358  87990        23182     11          10.1    18.900000      1   
141359  87991        21507     22           8.0    43.099998      3   
141360  87992        20287      9          10.7    49.700001      3   
141361  87993        20197      2           9.1    15.000000      1   

       is_apartment studio  total_area        price  build_year  \
0             false  false  155.600006   49900

In [10]:
data.dtypes

id                     int64
building_id            int64
floor                  int64
kitchen_area         float64
living_area          float64
rooms                  int64
is_apartment          object
studio                object
total_area           float64
price                float64
build_year             int64
building_type_int      int64
latitude             float64
longitude            float64
ceiling_height       float64
flats_count            int64
floors_total           int64
has_elevator          object
dtype: object

In [11]:
cat_features = data.select_dtypes(include='object')

In [12]:
cat_features.value_counts()

is_apartment  studio  has_elevator
false         false   true            125552
                      false            14438
true          false   true              1304
                      false               68
Name: count, dtype: int64

In [13]:
num_cols = data.select_dtypes(include='float')

In [14]:
num_cols.columns.to_list()

['kitchen_area',
 'living_area',
 'total_area',
 'price',
 'latitude',
 'longitude',
 'ceiling_height']

In [16]:
type(data.select_dtypes(include='float').columns.to_list())

list